In [ ]:
# Document scanner
# we are changing the prospective
# we will need points
#points of the billboard where you want to map points
# Bitwise operator to apply our img onto the bill board
import cv2
import numpy as np
# define the mouse callback function, will be needed lower
def onClick(event, x, y, flags, param):
    #we can check whether we selected the pts
    if event == cv2.EVEN_LBUTTONDOWN:
        if len(dst_points) < 4:
            #why 4 pts because we are changing the perspective, in 3 pts u can just scale and rotate
            dst_points.append([x,y])
            cv2.circle(img_copy, (x,y), 50, [0, 0, 255], -1) #to check whether we pointed or not
            cv2.imshow('Base Img', img_copy)
            

# load 2 images
base_img = cv2.imread('logo.png')
img_copy = base_img.copy()
# why do we copy it, because we want to draw circles on the img so we need a copie of it
img2 = cv2.imread('gerry.png')

# now we need to creat starting points and map it to destination points
# get the image data
# which are the starting img of our img
# we will use all the corners in the img
# top left position(0,0), top right(0, #of cols) ...
base_h, base_w = base_img.shape[:2] #took first 2 values of dimension, 3 values is # of channel img but we do not need it
img2_h, img2_w = img2.shape[:2] #same for the second img
# now we can creat src and dest pts
src_points = np.array(
    [
        #we will put 4 starting pts
        [0,0],
        [0, img2_h],
        [img2_w, img2_h],
        [img2_w, 0]
        #if you use counter-clock wise choosing pts at src pts, you need to choose in the same way in destination points, otherwise it will be incorrect mapping    
    ], dtype=np.float32
)
#since we will get the points with mouse
dst_points = []

#define the window for getting destination points
#imshow() opencv creates window with title and etc
#if we want to use call back in our window the we need to do following
cv2.namedWindow('Base Img', cv2.WINDOW_KEEPRATIO)
#in any kind of manipulations of resizing it will keep ratio the same sqr will be sqr rectangle will be alway rectangle
# in windows it is buggy
#We can assign call back function, it is called when smth happens in this window
#it will check if we do smth in the window
cv2.setMouseCallback('Base Img', onClick)

#show the img in order to br circled
cv2.imshow('Base Img', base_img)
cv2.waitKey(0)

#We need to point src pt to dest pts. to do that we use homography matrix
#we know that if we have 2 sets pts we can do inverse process 
dst_float = np.array(dst_points, dtype = np.float32)
H = cv2.getPerspectiveTransform(src_points, dst_float) #provides transformation matrix purpose is set of transformation to apply to src pts to get dest pts
#matrix to transform up line
#apply the homography matrix to the image angles
warped = cv2.warpPerspective(img2, H, (base_w, base_h))
#adwaited fnct gets as arg 2 imgs it shows contents of 2 imgs according to specified coefficient put
#with bitwise operator we can get rid of pixels we do not need
#with bitwise operator the concept of mask (binary masks) used to take needed portion of imgs. 
#create such mask
#empty mask with all black pxls
mask = np.zeros(base_img.shape, dtype.unit8) #our mask shouyld be of the same size as our img, dtype must be from 0 to 255

#now we can set to white all pxls inside of 4 pts

cv2.fillConvexPolly(mask, np.int32(dst_float), (255, 255, 255))
#apply the mask to the billboard img
#first invert the mask removing colored pixels
#billboard img
mask = cv2.bitwise_not(mask)
masked_bill = cv2.bitwise_and(base_img, mask)
#Apply the mask to our second img
final_img = cv2.bitwise_or(masked_bill, warped)

cv2.namedWindow('Final Img', cv2.WINDOW_KEEPRATIO)
cv2.imshow('final img', final_img)
cv2.waitKey(0)
